In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:



# args = {'output_dir': 'permuted_mnist_results',
#         'output_name': 'permuted_mnist_sorted',
#         'input_dir':'permuted_mnist_results',
#         'epochs': 20,
#         'input_fname_args':
#         {
#             'dataset': 'permuted_mnist',
#             'no_droput': False,
#             'sorting_file': 'none',
#             'remove_n': 0,
#             'keep_lowest_n': 0

#         }

#         }



!pwd
!ls
%cd drive
%cd MyDrive

/content
drive  sample_data
/content/drive
/content/drive/MyDrive


In [3]:
!git clone https://github.com/mtoneva/example_forgetting.git

Cloning into 'example_forgetting'...
remote: Enumerating objects: 194, done.
remote: Total 194 (delta 0), reused 0 (delta 0), pack-reused 194
Receiving objects: 100% (194/194), 566.18 KiB | 6.82 MiB/s, done.
Resolving deltas: 100% (102/102), done.


In [4]:
%cd example_forgetting/

/content/drive/MyDrive/example_forgetting


In [5]:
!pip install -r requirements.txt


     |████████████████████████████████| 519.5 MB 22 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.10.0+cu102
    Uninstalling torchvision-0.10.0+cu102:
      Successfully uninstalled torchvision-0.10.0+cu102
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 0.4.1.post2 which is incompatible.
fastai 1.0.61 requires torch>=1.0.0, but you have torch 0.4.1.post2 which is incompatible.


In [6]:
import pdb
import argparse
import numpy as np
import numpy.random as npr
import time
import os
import sys
import pickle

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.backends.cudnn as cudnn
from torch.optim.lr_scheduler import MultiStepLR

from torchvision.utils import make_grid
from torchvision import datasets, transforms

from Cutout.util.misc import CSVLogger
from Cutout.util.cutout import Cutout

from Cutout.model.resnet import ResNet18
from Cutout.model.wide_resnet import WideResNet



In [7]:

# Format time for printing purposes
def get_hms(seconds):
    m, s = divmod(seconds, 60)
    h, m = divmod(m, 60)

    return h, m, s


In [8]:

# Introduce Gaussian noise to noise_percentage of image pixels
def noisy(image, noise_percentage, noise_std):
    row, col, ch = image.shape
    num_corrupt = int(np.floor(noise_percentage * row * col / 100))

    # Randomly choose pixels to add noise to
    xy_coords = np.random.choice(row * col, num_corrupt, replace=False)
    chan_coords = np.random.choice(ch, num_corrupt, replace=True)
    xy_coords = np.unravel_index(xy_coords, (row, col))

    out = np.copy(image)

    mean = 120

    # Add randomly generated Gaussian noise to pixels
    for coord in range(num_corrupt):
        noise = np.random.normal(mean, noise_std, 1)
        out[xy_coords[0][coord], xy_coords[1][coord],
            chan_coords[coord]] += noise

    return out



In [18]:
#args
# Train model for one epoch
#
# example_stats: dictionary containing statistics accumulated over every presentation of example
#
def train(args, model, device, trainset, model_optimizer, epoch,
          example_stats):
    train_loss = 0.
    correct = 0.
    total = 0.

    model.train()

    # Get permutation to shuffle trainset
    trainset_permutation_inds = npr.permutation(
        np.arange(len(trainset.train_labels)))

    print('\n=> Training Epoch #%d' % (epoch))

    batch_size = args['batch_size']
    for batch_idx, batch_start_ind in enumerate(
            range(0, len(trainset.train_labels), batch_size)):

        # Get trainset indices for batch
        batch_inds = trainset_permutation_inds[batch_start_ind:
                                               batch_start_ind + batch_size]

        # Get batch inputs and targets, transform them appropriately
        transformed_trainset = []
        for ind in batch_inds:
            transformed_trainset.append(trainset.__getitem__(ind)[0])
        inputs = torch.stack(transformed_trainset)
        targets = torch.LongTensor(
            np.array(trainset.train_labels)[batch_inds].tolist())

        # Map to available device
        inputs, targets = inputs.to(device), targets.to(device)

        # Forward propagation, compute loss, get predictions
        model_optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        _, predicted = torch.max(outputs.data, 1)

        # Update statistics and loss
        acc = predicted == targets
        for j, index in enumerate(batch_inds):

            # Get index in original dataset (not sorted by forgetting)
            index_in_original_dataset = train_indx[index]

            # Compute missclassification margin
            output_correct_class = outputs.data[j, targets[j].item()]
            sorted_output, _ = torch.sort(outputs.data[j, :])
            if acc[j]:
                # Example classified correctly, highest incorrect class is 2nd largest output
                output_highest_incorrect_class = sorted_output[-2]
            else:
                # Example misclassified, highest incorrect class is max output
                output_highest_incorrect_class = sorted_output[-1]
            margin = output_correct_class.item(
            ) - output_highest_incorrect_class.item()

            # Add the statistics of the current training example to dictionary
            index_stats = example_stats.get(index_in_original_dataset,
                                            [[], [], []])
            index_stats[0].append(loss[j].item())
            index_stats[1].append(acc[j].sum().item())
            index_stats[2].append(margin)
            example_stats[index_in_original_dataset] = index_stats

        # Update loss, backward propagate, update optimizer
        loss = loss.mean()
        train_loss += loss.item()
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()
        loss.backward()
        model_optimizer.step()

        sys.stdout.write('\r')
        sys.stdout.write(
            '| Epoch [%3d/%3d] Iter[%3d/%3d]\t\tLoss: %.4f Acc@1: %.3f%%' %
            (epoch, args['epochs'], batch_idx + 1,
             (len(trainset) // batch_size) + 1, loss.item(),
             100. * correct.item() / total))
        sys.stdout.flush()

        # Add training accuracy to dict
        index_stats = example_stats.get('train', [[], []])
        index_stats[1].append(100. * correct.item() / float(total))
        example_stats['train'] = index_stats


In [19]:
#args
# Evaluate model predictions on heldout test data
#
# example_stats: dictionary containing statistics accumulated over every presentation of example
#
def test(epoch, model, device, example_stats):
    global best_acc
    test_loss = 0.
    correct = 0.
    total = 0.
    test_batch_size = 32

    model.eval()

    for batch_idx, batch_start_ind in enumerate(
            range(0, len(test_dataset.test_labels), test_batch_size)):

        # Get batch inputs and targets
        transformed_testset = []
        for ind in range(
                batch_start_ind,
                min(
                    len(test_dataset.test_labels),
                    batch_start_ind + test_batch_size)):
            transformed_testset.append(test_dataset.__getitem__(ind)[0])
        inputs = torch.stack(transformed_testset)
        targets = torch.LongTensor(
            np.array(
                test_dataset.test_labels)[batch_start_ind:batch_start_ind +
                                          test_batch_size].tolist())

        # Map to available device
        inputs, targets = inputs.to(device), targets.to(device)

        # Forward propagation, compute loss, get predictions
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss = loss.mean()
        test_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()

    # Add test accuracy to dict
    acc = 100. * correct.item() / total
    index_stats = example_stats.get('test', [[], []])
    index_stats[1].append(100. * correct.item() / float(total))
    example_stats['test'] = index_stats
    print("\n| Validation Epoch #%d\t\t\tLoss: %.4f Acc@1: %.2f%%" %
          (epoch, loss.item(), acc))

    # Save checkpoint when best model
    if acc > best_acc:
        print('| Saving Best model...\t\t\tTop1 = %.2f%%' % (acc))
        state = {
            'acc': acc,
            'epoch': epoch,
        }
        save_point = os.path.join(args['output_dir'], 'checkpoint', args['dataset'])
        os.makedirs(save_point, exist_ok=True)
        torch.save(state, os.path.join(save_point, save_fname + '.t7'))
        best_acc = acc



In [11]:

model_options = ['resnet18', 'wideresnet']
dataset_options = ['cifar10', 'cifar100']


In [12]:
args = {'dataset': 'cifar100',
        'model':'resnet18',
        'batch_size': 128,
        'epochs':20,
        'learning_rate':0.01,
        'data_augmentation':False,
        'cutout':False,
        'n_holes':1,
        'length':16,
        'no_cuda':False,
        'seed':2,
        'sorting_file':"none",
        'remove_n':0,
        'keep_lowest_n':0,
        'no_dropout':False,
        'remove_subsample':0,
        'noise_percent_labels':0,
        'noise_percent_pixels':0,
        'noise_std_pixels':0,
        'optimizer':'sgd',
        'input_dir':'cifar100_results/',
        'output_dir':'cifar100_results/'

        }

# Enter all arguments that you want to be in the filename of the saved output
ordered_args = [
    'dataset', 'data_augmentation', 'cutout', 'seed', 'sorting_file',
    'remove_n', 'keep_lowest_n', 'remove_subsample', 'noise_percent_labels',
    'noise_percent_pixels', 'noise_std_pixels'
]
# # Parse arguments and setup name of output file with forgetting stats
# args = parser.parse_args()
args_dict = args
# print(args_dict)
save_fname = '__'.join(
    '{}_{}'.format(arg, args_dict[arg]) for arg in ordered_args)

# Set appropriate devices
args['cuda'] = not args['no_cuda'] and torch.cuda.is_available()
use_cuda = args['cuda']
device = torch.device("cuda" if use_cuda else "cpu")
cudnn.benchmark = True  # Should make training go faster for large models

# Set random seed for initialization
torch.manual_seed(args['seed'])
if args['cuda']:
    torch.cuda.manual_seed(args['seed'])
npr.seed(args['seed'])

# Image Preprocessing
normalize = transforms.Normalize(
    mean=[x / 255.0 for x in [125.3, 123.0, 113.9]],
    std=[x / 255.0 for x in [63.0, 62.1, 66.7]])

# Setup train transforms
train_transform = transforms.Compose([])
if args['data_augmentation']:
    train_transform.transforms.append(transforms.RandomCrop(32, padding=4))
    train_transform.transforms.append(transforms.RandomHorizontalFlip())
train_transform.transforms.append(transforms.ToTensor())
train_transform.transforms.append(normalize)
if args['cutout']:
    train_transform.transforms.append(
        Cutout(n_holes=args['n_holes'], length=args['length']))


In [13]:

# Setup test transforms
test_transform = transforms.Compose([transforms.ToTensor(), normalize])

os.makedirs(args['output_dir'], exist_ok=True)
!wget https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
!tar -zxvf cifar-10-python.tar.gz
!wget https://www.cs.toronto.edu/~kriz/cifar-100-python.tar.gz
!tar -zxvf cifar-100-python.tar.gz

from torchvision.datasets import CIFAR10
from torchvision.datasets import CIFAR100
from torchvision import transforms


if args['dataset'] == 'cifar10':
  num_classes=10
  train_dataset = CIFAR10('./', download=True,
  transform=transforms.Compose([
  transforms.ToTensor(),
  ]), train=True)


  test_dataset = CIFAR10('./', download=True,
  transform=transforms.Compose([
  transforms.ToTensor(),
  ]), train=False)

elif args['dataset'] == 'cifar100':
  num_classes=100
  train_dataset = CIFAR100('./', download=True,
  transform=transforms.Compose([
  transforms.ToTensor(),
  ]), train=True)


  test_dataset = CIFAR100('./', download=True,
  transform=transforms.Compose([
  transforms.ToTensor(),
  ]), train=False)



--2021-09-10 07:37:51--  https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
Resolving www.cs.toronto.edu (www.cs.toronto.edu)... 128.100.3.30
Connecting to www.cs.toronto.edu (www.cs.toronto.edu)|128.100.3.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 170498071 (163M) [application/x-gzip]
Saving to: ‘cifar-10-python.tar.gz’

cifar-10-python.tar 100%[===================>] 162.60M  29.0MB/s    in 6.8s    

2021-09-10 07:37:58 (24.0 MB/s) - ‘cifar-10-python.tar.gz’ saved [170498071/170498071]

cifar-10-batches-py/
cifar-10-batches-py/data_batch_4
cifar-10-batches-py/readme.html
cifar-10-batches-py/test_batch
cifar-10-batches-py/data_batch_3
cifar-10-batches-py/batches.meta
cifar-10-batches-py/data_batch_2
cifar-10-batches-py/data_batch_5
cifar-10-batches-py/data_batch_1
--2021-09-10 07:38:02--  https://www.cs.toronto.edu/~kriz/cifar-100-python.tar.gz
Resolving www.cs.toronto.edu (www.cs.toronto.edu)... 128.100.3.30
Connecting to www.cs.toronto.edu (www.

In [15]:

# Get indices of examples that should be used for training
if args['sorting_file'] == 'none':
    train_indx = np.array(range(len(train_dataset.train_labels)))
else:
    try:
        with open(
                os.path.join(args['input_dir'], args['sorting_file']) + '.pkl',
                'rb') as fin:
            ordered_indx = pickle.load(fin)['indices']
    except IOError:
        with open(os.path.join(args['input_dir'], args['sorting_file']),
                  'rb') as fin:
            ordered_indx = pickle.load(fin)['indices']

    # Get the indices to remove from training
    elements_to_remove = np.array(
        ordered_indx)[args['keep_lowest_n']:args['keep_lowest_n'] + args['remove_n']]

    # Remove the corresponding elements
    train_indx = np.setdiff1d(
        range(len(train_dataset.train_labels)), elements_to_remove)

if args['keep_lowest_n'] < 0:
    # Remove remove_n number of examples from the train set at random
    train_indx = npr.permutation(np.arange(len(
        train_dataset.train_labels)))[:len(train_dataset.train_labels) -
                                      args['remove_n']]

elif args['remove_subsample']:
    # Remove remove_sample number of examples at random from the first keep_lowest_n examples
    # Useful when the first keep_lowest_n examples have equal forgetting counts
    lowest_n = np.array(ordered_indx)[0:args['keep_lowest_n']]
    train_indx = lowest_n[npr.permutation(np.arange(
        args['keep_lowest_n']))[:args['keep_lowest_n'] - args['remove_subsample']]]
    train_indx = np.hstack((train_indx,
                            np.array(ordered_indx)[args['keep_lowest_n']:]))


In [17]:
#args
# Reassign train data and labels
train_dataset.train_data = train_dataset.train_data[train_indx, :, :, :]
train_dataset.train_labels = np.array(
    train_dataset.train_labels)[train_indx].tolist()

# Introduce noise to images if specified
if args['noise_percent_pixels']:
    for ind in range(len(train_indx)):
        image = train_dataset.train_data[ind, :, :, :]
        noisy_image = noisy(image, args['noise_percent_pixels'], args['noise_std_pixels'])
        train_dataset.train_data[ind, :, :, :] = noisy_image

# Introduce noise to labels if specified
if args['noise_percent_labels']:
    fname = os.path.join(args['output_dir'], save_fname)

    with open(fname + "_changed_labels.txt", "w") as f:

        # Compute number of labels to change
        nlabels = len(train_dataset.train_labels)
        nlabels_to_change = int(args['noise_percent_labels'] * nlabels / 100)
        ntrain_labels = len(np.unique(train_dataset.train_labels))
        print('flipping ' + str(nlabels_to_change) + ' labels')

        # Randomly choose which labels to change, get indices
        labels_inds_to_change = npr.choice(
            np.arange(nlabels), nlabels_to_change, replace=False)

        # Flip each of the randomly chosen labels
        for l, label_ind_to_change in enumerate(labels_inds_to_change):

            # Possible choices for new label
            label_choices = np.arange(ntrain_labels)

            # Get true label to remove it from the choices
            true_label = train_dataset.train_labels[label_ind_to_change]

            # Remove true label from choices
            label_choices = np.delete(
                label_choices,
                true_label)  # the label is the same as the index of the label

            # Get new label and relabel the example with it
            noisy_label = npr.choice(label_choices, 1)
            train_dataset.train_labels[label_ind_to_change] = noisy_label[0]

            # Write the example index from the original example order, the old, and the new label
            f.write(
                str(train_indx[label_ind_to_change]) + ' ' + str(true_label) +
                ' ' + str(noisy_label[0]) + '\n')


In [21]:
print('Training on ' + str(len(train_dataset.train_labels)) + ' examples')

# Setup model
if args['model'] == 'resnet18':
    model = ResNet18(num_classes=num_classes)
elif args['model'] == 'wideresnet':
    if args['dataset'] == 'svhn':
        model = WideResNet(
            depth=16, num_classes=num_classes, widen_factor=8, dropRate=0.4)
    else:
        model = WideResNet(
            depth=28, num_classes=num_classes, widen_factor=10, dropRate=0.3)
else:
    print(
        'Specified model not recognized. Options are: resnet18 and wideresnet')

# Setup loss
model = model.cuda()
criterion = nn.CrossEntropyLoss().cuda()
criterion.__init__(reduce=False)

# Setup optimizer
if args['optimizer'] == 'adam':
    model_optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
elif args['optimizer'] == 'sgd':
    model_optimizer = torch.optim.SGD(
        model.parameters(),
        lr=args['learning_rate'],
        momentum=0.9,
        nesterov=True,
        weight_decay=5e-4)
    scheduler = MultiStepLR(
        model_optimizer, milestones=[60, 120, 160], gamma=0.2)
else:
    print('Specified optimizer not recognized. Options are: adam and sgd')


Training on 50000 examples


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


In [22]:

# Initialize dictionary to save statistics for every example presentation
example_stats = {}

best_acc = 0
elapsed_time = 0
for epoch in range(20):
    start_time = time.time()

    train(args, model, device, train_dataset, model_optimizer, epoch, example_stats)
    test(epoch, model, device, example_stats)

    epoch_time = time.time() - start_time
    elapsed_time += epoch_time
    print('| Elapsed time : %d:%02d:%02d' % (get_hms(elapsed_time)))

    # Update optimizer step
    if args['optimizer'] == 'sgd':
        scheduler.step(epoch)

    # Save the stats dictionary
    fname = os.path.join(args['output_dir'], save_fname)
    with open(fname + "__stats_dict.pkl", "wb") as f:
        pickle.dump(example_stats, f)

    # Log the best train and test accuracy so far
    with open(fname + "__best_acc.txt", "w") as f:
        f.write('train test \n')
        f.write(str(max(example_stats['train'][1])))
        f.write(' ')
        f.write(str(max(example_stats['test'][1])))


=> Training Epoch #0
| Epoch [  0/ 20] Iter[391/391]		Loss: 3.2934 Acc@1: 15.098%
| Validation Epoch #0			Loss: 3.4130 Acc@1: 18.29%
| Saving Best model...			Top1 = 18.29%
| Elapsed time : 0:01:57

=> Training Epoch #1
| Epoch [  1/ 20] Iter[391/391]		Loss: 2.3406 Acc@1: 31.256%
| Validation Epoch #1			Loss: 2.6277 Acc@1: 17.09%
| Elapsed time : 0:03:50

=> Training Epoch #2
| Epoch [  2/ 20] Iter[391/391]		Loss: 1.8912 Acc@1: 42.278%
| Validation Epoch #2			Loss: 2.8057 Acc@1: 33.67%
| Saving Best model...			Top1 = 33.67%
| Elapsed time : 0:05:43

=> Training Epoch #3
| Epoch [  3/ 20] Iter[391/391]		Loss: 1.8820 Acc@1: 50.368%
| Validation Epoch #3			Loss: 1.6283 Acc@1: 46.86%
| Saving Best model...			Top1 = 46.86%
| Elapsed time : 0:07:36

=> Training Epoch #4
| Epoch [  4/ 20] Iter[391/391]		Loss: 1.3607 Acc@1: 57.482%
| Validation Epoch #4			Loss: 2.0550 Acc@1: 42.95%
| Elapsed time : 0:09:29

=> Training Epoch #5
| Epoch [  5/ 20] Iter[391/391]		Loss: 1.2283 Acc@1: 63.826%
| Val

In [23]:
# python order_examples_by_forgetting.py 
#     --output_dir [cifar10/cifar100]_results 
#     --output_name [cifar10/cifar100]_sorted 
#     --input_dir [cifar10/cifar100]_results 
#     --input_fname_args 
#             dataset [cifar10/cifar100]
#             data_augmentation True 
#             cutout False 
#             sorting_file none 
#             remove_n 0 
#             keep_lowest_n 0 
#             remove_subsample 0 
#             noise_percent_labels 0 
#             noise_percent_pixels 0 
#             noise_std_pixels 0
import argparse
import numpy as np
import os
import pickle



In [24]:
#args

def compute_forgetting_statistics(diag_stats, npresentations):

    presentations_needed_to_learn = {}
    unlearned_per_presentation = {}
    margins_per_presentation = {}
    first_learned = {}

    for example_id, example_stats in diag_stats.items():

        # Skip 'train' and 'test' keys of diag_stats
        if not isinstance(example_id, str):

            # Forgetting event is a transition in accuracy from 1 to 0
            presentation_acc = np.array(example_stats[1][:npresentations])
            transitions = presentation_acc[1:] - presentation_acc[:-1]

            # Find all presentations when forgetting occurs
            if len(np.where(transitions == -1)[0]) > 0:
                unlearned_per_presentation[example_id] = np.where(
                    transitions == -1)[0] + 2
            else:
                unlearned_per_presentation[example_id] = []

            # Find number of presentations needed to learn example, 
            # e.g. last presentation when acc is 0
            if len(np.where(presentation_acc == 0)[0]) > 0:
                presentations_needed_to_learn[example_id] = np.where(
                    presentation_acc == 0)[0][-1] + 1
            else:
                presentations_needed_to_learn[example_id] = 0

            # Find the misclassication margin for each presentation of the example
            margins_per_presentation = np.array(
                example_stats[2][:npresentations])

            # Find the presentation at which the example was first learned, 
            # e.g. first presentation when acc is 1
            if len(np.where(presentation_acc == 1)[0]) > 0:
                first_learned[example_id] = np.where(
                    presentation_acc == 1)[0][0]
            else:
                first_learned[example_id] = np.nan

    return presentations_needed_to_learn, unlearned_per_presentation, margins_per_presentation, first_learned


# Sorts examples by number of forgetting counts during training, in ascending order
# If an example was never learned, it is assigned the maximum number of forgetting counts
# If multiple training runs used, sort examples by the sum of their forgetting counts over all runs
#
# unlearned_per_presentation_all: list of dictionaries, one per training run
# first_learned_all: list of dictionaries, one per training run
# npresentations: number of training epochs
#
# Returns 2 numpy arrays containing the sorted example ids and corresponding forgetting counts
#
def sort_examples_by_forgetting(unlearned_per_presentation_all,
                                first_learned_all, npresentations):

    # Initialize lists
    example_original_order = []
    example_stats = []

    for example_id in unlearned_per_presentation_all[0].keys():

        # Add current example to lists
        example_original_order.append(example_id)
        example_stats.append(0)

        # Iterate over all training runs to calculate the total forgetting count for current example
        for i in range(len(unlearned_per_presentation_all)):

            # Get all presentations when current example was forgotten during current training run
            stats = unlearned_per_presentation_all[i][example_id]

            # If example was never learned during current training run, add max forgetting counts
            if np.isnan(first_learned_all[i][example_id]):
                example_stats[-1] += npresentations
            else:
                example_stats[-1] += len(stats)

    print('Number of unforgettable examples: {}'.format(
        len(np.where(np.array(example_stats) == 0)[0])))
    return np.array(example_original_order)[np.argsort(
        example_stats)], np.sort(example_stats)


# Checks whether a given file name matches a list of specified arguments
#
# fname: string containing file name
# args_list: list of strings containing argument names and values, i.e. [arg1, val1, arg2, val2,..]
#
# Returns 1 if filename matches the filter specified by the argument list, 0 otherwise
#
def check_filename(fname, args_list):

    # # If no arguments are specified to filter by, pass filename
    # if args_list is None:
    #     return 1

    # for arg_ind in list(args_list):#np.arange(0, len(args_list), 2):
    #     arg = str(arg_ind)
    #     arg_value = str(args_list[arg_ind])

    #     # Check if filename matches the current arg and arg value
    #     if arg + '_' + arg_value + '__' not in fname:
    #         print('skipping file: ' + fname)
    #         return 0

    return 1



In [25]:


args = {'output_dir': 'cifar100_results',
        'output_name': 'cifar100_sorted',
        'input_dir':'cifar100_results',
        'epochs': 20,
        'input_fname_args':
        {
            'dataset': 'cifar100',
            'data_augmentation': True,
            'cutout': False,
            'no_droput': False,
            'sorting_file': 'none',
            'remove_n': 0,
            'keep_lowest_n': 0,
            'remove_subsample':0,
            'noise_percent_labels':0,
            'noise_percent_pixels':0,
            'noise_std_pixels':0

        }

        }

# Initialize lists to collect forgetting stastics per example across multiple training runs
unlearned_per_presentation_all, first_learned_all = [], []

for d, _, fs in os.walk(args['input_dir']):
    for f in fs:

        # Find the files that match input_fname_args and compute forgetting statistics
        if f.endswith('stats_dict.pkl') and check_filename(
                f, args['input_fname_args']):
            print('including file: ' + f)

            # Load the dictionary compiled during training run
            with open(os.path.join(d, f), 'rb') as fin:
                loaded = pickle.load(fin)

            # Compute the forgetting statistics per example for training run
            _, unlearned_per_presentation, _, first_learned = compute_forgetting_statistics(
                loaded, args['epochs'])

            unlearned_per_presentation_all.append(
                unlearned_per_presentation)
            first_learned_all.append(first_learned)

if len(unlearned_per_presentation_all) == 0:
    print('No input files found in {} that match {}'.format(
        args['input_dir'], args['input_fname_args']))
else:

    # Sort examples by forgetting counts in ascending order, over one or more training runs
    ordered_examples, ordered_values = sort_examples_by_forgetting(
        unlearned_per_presentation_all, first_learned_all, args['epochs'])

    # Save sorted output
    if args['output_name'].endswith('.pkl'):
        with open(os.path.join(args['output_dir'], args['output_name']),
                  'wb') as fout:
            pickle.dump({
                'indices': ordered_examples,
                'forgetting counts': ordered_values
            }, fout)
    else:
        with open(
                os.path.join(args['output_dir'], args['output_name'] + '.pkl'),
                'wb') as fout:
            pickle.dump({
                'indices': ordered_examples,
                'forgetting counts': ordered_values
            }, fout)

including file: dataset_cifar100__data_augmentation_False__cutout_False__seed_2__sorting_file_none__remove_n_0__keep_lowest_n_0__remove_subsample_0__noise_percent_labels_0__noise_percent_pixels_0__noise_std_pixels_0__stats_dict.pkl
Number of unforgettable examples: 22585
